In [5]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

base = "https://www.basketball-reference.com/teams/"
base2 = "https://www.basketball-reference.com/leagues/NBA_"
start_year = 2000
end_year = 2021
numTeams = 30
numCols = 13
teamNames = {}
bbrefStats = {0: 'g', 1: 'fg_pct', 2: 'fg3_pct', 3: 'efg_pct', 4: 'ft_pct', 5: 'trb_per_g', 6: 'ast_per_g', \
             7: 'stl_per_g', 8: 'blk_per_g', 9: 'tov_per_g', 10:'pts_per_g'}
teamTitles = {0: 'Atlanta Hawks', 1: 'Boston Celtics', 2: 'Brooklyn Nets', 3: 'Charlotte Hornets', 4: \
              'Chicago Bulls', 5: 'Cleveland Cavaliers', 6: 'Dallas Mavericks', 7: 'Denver Nuggets', 8: \
             'Detroit Pistons', 9: 'Golden State Warriors', 10: 'Houston Rockets', 11: 'Indiana Pacers', 12: \
             'Los Angeles Clippers', 13: 'Los Angeles Lakers', 14: 'Memphis Grizzlies', 15: 'Miami Heat', 16: \
             'Milwaukee Bucks', 17: 'Minnesota Timberwolves', 18: 'New Orleans Pelicans', 19: \
             'New York Knicks', 20: 'Oklahoma City Thunder', 21: 'Orlando Magic', 22: 'Philadelphia 76ers', 23: \
             'Phoenix Suns', 24: 'Portland Trail Blazers', 25: 'Sacramento Kings', 26: 'San Antonio Spurs', 27: \
             'Toronto Raptors', 28: 'Utah Jazz', 29: 'Washington Wizards'}
teamNames = {0: 'ATL/', 1: 'BOS/', 2: 'BRK/', 3: 'CHA/', 4: 'CHI/', 5: 'CLE/', 6: 'DAL/', 7: 'DEN/', 8: 'DET/', \
             9: 'GSW/', 10: 'HOU/', 11: 'IND/', 12: 'LAC/', 13: 'LAL/', 14: 'MEM/', 15: 'MIA/', 16: 'MIL/', \
             17: 'MIN/', 18: 'NOP/', 19: 'NYK/', 20: 'OKC/', 21: 'ORL/', 22: 'PHI/', 23: 'PHO/', 24: 'POR/', \
             25: 'SAC/', 26: 'SAS/', 27: 'TOR/', 28: 'UTA/', 29: 'WAS/'}
df = []

''' Want to include Name, Games Played, FG%, 3P% (if > 1), eFG%, FT%, Rebounds, Assists, Steals, Blocks, Turnovers, 
Points, Team Seed, Team Seed Last Year '''

year = 1
html = ".html"

while year < end_year - start_year + 1:
    teams = []
    teamNum = 0
    
    # Constructing the table of standings for current season and past season
    URL2 = base2 + str(year+start_year) + "_standings" + html
    URL3 = base2 + str(year+start_year-1) + "_standings" + html
    soup2 = BeautifulSoup(requests.get(URL2).content, 'html.parser')
    soup3 = BeautifulSoup(requests.get(URL3).content, 'html.parser')
    standingsCurrent2 = soup2.find(id='all_expanded_standings')
    standingsPast3 = soup3.find(id='all_expanded_standings')
    standingBig2 = None
    standingBig3 = None

    i = 0
    for standing in standingsCurrent2:
        if i == 4:
            standingBig2 = standing
        i += 1

    i = 0
    for standing in standingsPast3:
        if i == 4:
            standingBig3 = standing
        i += 1

    soup2 = BeautifulSoup(standingBig2, 'html.parser')
    soup3 = BeautifulSoup(standingBig3, 'html.parser')
    standingsCurrent = soup2.find('tbody').find_all('tr')
    standingsPast = soup3.find('tbody').find_all('tr')

    while teamNum < numTeams:
        currentYear = year + start_year
        print(str(currentYear) + " " + str(teamNum))

        # Ensuring name changes and relocations don't get in the way
        if teamNum == 2 and currentYear <= 2012:
            URL = base + 'NJN/' + str(currentYear) + html
        elif teamNum == 3:
            if currentYear <= 2002:
                URL = base + 'CHH/' + str(currentYear) + html
            elif currentYear >= 2003 and currentYear <= 2004:
                teamNum += 1
                continue
            elif currentYear >= 2015:
                URL = base + 'CHO/' + str(currentYear) + html
        elif teamNum == 14 and currentYear == 2001:
            URL = base + 'VAN/' + str(currentYear) + html
        elif teamNum == 18:
            if (currentYear >= 2003 and currentYear <= 2005) or (currentYear >= 2008 and currentYear <= 2013):
                URL = base + 'NOH/' + str(currentYear) + html
            elif currentYear >= 2006 and currentYear <= 2007:
                URL = base + 'NOK/' + str(currentYear) + html
            elif currentYear < 2003:
                teamNum += 1
                continue
        elif teamNum == 20 and currentYear <= 2008:
            URL = base + 'SEA/' + str(currentYear) + html
        else:
            URL = base + teamNames[teamNum] + str(currentYear) + html

        # Arranging the data to be scraped
        soup = BeautifulSoup(requests.get(URL).content, 'html.parser')
        table = soup.find(id='div_per_game').find('tbody')
        players = table.find_all('tr')
        team = pd.DataFrame(np.zeros((len(players), numCols)))

        # Getting the differential between current and past seeds (same for all players on one team)
        j = 0
        currentSeed = 0
        pastSeed = 0
        while j < len(standingsCurrent):
            if standingsCurrent[j].find('td', class_='left').text == teamTitles[teamNum]:
                currentSeed = j + 1
            if standingsPast[j].find('td', class_='left').text == teamTitles[teamNum]:
                pastSeed = j + 1
            if len(standingsCurrent) != len(standingsPast) and j == 28:
                if currentSeed == 0:
                    currentSeed = 29
                break
            j += 1
        seedDiff = pastSeed - currentSeed

        # Scraping the other features
        i = 0
        for player in players:
            team.loc[i, 0] = player.find('td', class_='left').text # Names
            stats = player.find_all('td', class_='right')
            col = 1
            j = 0
            while j < len(stats):
                stat = 0
                currentStat = stats[j].get('data-stat')
                while stat < len(bbrefStats):
                    if currentStat == 'fg3a_per_g':
                        if float(stats[j].text) < 1:
                            col += 1
                        break
                    if currentStat == bbrefStats[stat]:
                        if col == 4 and currentStat == 'fg3_pct':
                            break
                        team.loc[i, col] = stats[j].text
                        col += 1
                        break
                    stat += 1
                j += 1
            team.loc[i, col] = currentSeed # Current Seed
            team.loc[i, col + 1] = seedDiff # Team Seed Last Season - Team Seed This Season
            i += 1

        teams.append(team)
        teamNum += 1
    
    df.append(teams)
    
    year += 1

df_final = pd.DataFrame()

for yearly in df:
    teams = pd.DataFrame()
    for team in yearly:
        teams = pd.concat([teams, team], ignore_index=True)
    teams.columns = ['Name', 'Games', 'FG%', '3P%', 'eFG%', 'FT%', 'Rebounds', 'Assists', 'Steals', 'Blocks', \
                   'Turnovers', 'Points', 'Seed', 'Impact']
    teams = teams.drop_duplicates(subset=['Name']) # Assuming players who change teams midway aren't MVP candidates
    df_final = pd.concat([df_final, teams], ignore_index=False)
    
compression_opts = dict(method='zip', archive_name='BBRefMVPData.csv')  
df_final.to_csv('BBRefMVPData.zip', compression=compression_opts)  

2001 0
2001 1
2001 2
2001 3
2001 4
2001 5
2001 6
2001 7
2001 8
2001 9
2001 10
2001 11
2001 12
2001 13
2001 14
2001 15
2001 16
2001 17
2001 18
2001 19
2001 20
2001 21
2001 22
2001 23
2001 24
2001 25
2001 26
2001 27
2001 28
2001 29
2002 0
2002 1
2002 2
2002 3
2002 4
2002 5
2002 6
2002 7
2002 8
2002 9
2002 10
2002 11
2002 12
2002 13
2002 14
2002 15
2002 16
2002 17
2002 18
2002 19
2002 20
2002 21
2002 22
2002 23
2002 24
2002 25
2002 26
2002 27
2002 28
2002 29
2003 0
2003 1
2003 2
2003 3
2003 4
2003 5
2003 6
2003 7
2003 8
2003 9
2003 10
2003 11
2003 12
2003 13
2003 14
2003 15
2003 16
2003 17
2003 18
2003 19
2003 20
2003 21
2003 22
2003 23
2003 24
2003 25
2003 26
2003 27
2003 28
2003 29
2004 0
2004 1
2004 2
2004 3
2004 4
2004 5
2004 6
2004 7
2004 8
2004 9
2004 10
2004 11
2004 12
2004 13
2004 14
2004 15
2004 16
2004 17
2004 18
2004 19
2004 20
2004 21
2004 22
2004 23
2004 24
2004 25
2004 26
2004 27
2004 28
2004 29
2005 0
2005 1
2005 2
2005 3
2005 4
2005 5
2005 6
2005 7
2005 8
2005 9
2005 10
20